## Neuron Simulations

Neurons have complex biophysical properties that can be simulated using python.
In the following I will present progressively more complex models of neuron simulation.
The project will be concluded by an algorithm that translates auditory information into electrical signals that activate neurons in the cochlea (inner ear). Its a rudimentary approach to a cochela implant algorithm

## The Leaky Integrate and Fire Model

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


def membrane_voltage(vm, i_stim, cm, g_leak, params):
    r = 1 / g_leak
    tau = cm * r

    for index in range(len(params["time"]) - 1):
        if vm[index] < params["v_thr"]:
            vm[index + 1] = (((-vm[index] + params["v_rest"]) / tau) + (i_stim[index] / cm)) * \
                            params["step_size"] + vm[index]
        elif vm[index] == params["v_spike"]:
            vm[index + 1] = params["v_rest"]
        elif vm[index] > params["v_thr"]:
            vm[index + 1] = params["v_spike"]

    return vm


# Define the params dictionary
params = {
    "v_rest": -70e-3,
    "v_thr": -30e-3,
    "v_spike": 20e-3,
    "step_size": 30e-6,
    "cm": 1e-6,
}

params["time"] = np.arange(0, 0.1, params["step_size"])

# Create an interactive plot using widgets.interact
i_stim_slider = widgets.FloatSlider(
    value=10e-6,
    min=3e-6,
    max=10e-6,
    step=1e-6,
    description='Stimulation current (A)',
    continuous_update=False
)

def interactive_plot(i_stim_val):
    vm = np.zeros(len(params["time"]))
    vm[0] = params["v_rest"]

    i_stim = np.zeros(len(params["time"]))
    i_stim[:4999] = i_stim_val

    g_leak = 100e-6

    vm = membrane_voltage(vm, i_stim, params["cm"], g_leak, params)

    plt.figure(figsize=(16, 6))
    plt.plot(params["time"], vm)
    plt.xlabel("Time (s)")
    plt.ylabel("Membrane Voltage (V)")
    plt.title("Leaky Integrated Fire Model")
    plt.grid()
    plt.show()

widgets.interactive(interactive_plot, i_stim_val=i_stim_slider)

interactive(children=(FloatSlider(value=1e-05, continuous_update=False, description='Stimulation current (A)',…